In [0]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

download = drive.CreateFile({'id': '1jPZAGYcuyJdvpuFXzX6yfiTV4qZKk_v1'})
download.GetContentFile('Data_Train.xlsx')
train_df = pd.read_excel('Data_Train.xlsx')

train_df.head()

download = drive.CreateFile({'id': '1fYtpRafrbg75Qgbn8eW_uiVIhieMqm21'})
download.GetContentFile('Data_Test.xlsx')
test_df = pd.read_excel('Data_Test.xlsx')

test_df.head()

     |████████████████████████████████| 993kB 2.8MB/s 
  Created wheel for PyDrive: filename=PyDrive-1.3.1-cp36-none-any.whl size=27435 sha256=2d14eee5a9aebe232a61fa09442b8dc411005fbc42e4fbc0b5d44bd908099139
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


,STORY
0,2019 will see gadgets like gaming smartphones ...
1,It has also unleashed a wave of changes in the...
2,It can be confusing to pick the right smartpho...
3,The mobile application is integrated with a da...
4,We have rounded up some of the gadgets that sh...


In [0]:
!pip install pytorch-pretrained-bert==0.4.0

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from fastai.text import * 
from fastai.callbacks import *
from pytorch_pretrained_bert.modeling import BertConfig, BertForSequenceClassification
from pytorch_pretrained_bert import BertTokenizer
from shutil import copyfile
from sklearn.model_selection import train_test_split


# nltk for preprocessing of text data
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# sklearn for preprocessing and machine learning models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import AdaBoostClassifier,VotingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier 

import warnings
warnings.filterwarnings("ignore")

     |████████████████████████████████| 51kB 2.0MB/s 
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
df = train_df
df['word_count'] = df['STORY'].str.split().map(len)
q = df[(df['word_count']>=200)]

In [0]:
test_df['word_count'] = test_df['STORY'].str.split().map(len)
p = test_df[(test_df['word_count']>=200)]

In [0]:
test_df.drop(['word_count'],axis=1,inplace=True)
df.drop(['word_count'],axis=1,inplace=True)

In [0]:
import gc
gc.collect()

29

In [0]:
df['Politics'] = df['SECTION'].apply(lambda x: 1. if x==0 else 0.)
df['Technology'] = df['SECTION'].apply(lambda x: 1. if x==1 else 0.)
df['Entertainment'] = df['SECTION'].apply(lambda x: 1. if x==2 else 0.)
df['Business'] = df['SECTION'].apply(lambda x: 1. if x==3 else 0.)

In [0]:
train = df[:int(len(df)*.8)]
valid = df[int(len(df)*.8):]

train.to_csv('train.csv',index_label=False )
valid.to_csv('valid.csv',index_label=False )

In [0]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)

config = Config(
    testing=False,
    bert_model_name="bert-large-uncased",
    max_lr=3e-5,
    epochs=4,
    use_fp16=True,
    bs=8,
    discriminative=False,
    max_seq_len=256,
)

In [0]:
bert_tok = BertTokenizer.from_pretrained(config.bert_model_name)

100%|██████████| 231508/231508 [00:00<00:00, 5603054.40B/s]


In [0]:
class FastAiBertTokenizer(BaseTokenizer): 
    """Wrapper around BertTokenizer to be compatible with fast.ai"""
    def __init__(self, tokenizer: BertTokenizer, max_seq_len: int=128, **kwargs): 
         self._pretrained_tokenizer = tokenizer 
         self.max_seq_len = max_seq_len 
    def __call__(self, *args, **kwargs): 
         return self 
    def tokenizer(self, t:str) -> List[str]: #Limits the maximum sequence length
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]

In [0]:
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=config.max_seq_len), pre_rules=[], post_rules=[])

In [0]:
def _join_texts(texts:Collection[str], mark_fields:bool=False, sos_token:Optional[str]=BOS):
    """Borrowed from fast.ai source"""
    if not isinstance(texts, np.ndarray): texts = np.array(texts)
    if is1d(texts): texts = texts[:,None]
    df = pd.DataFrame({i:texts[:,i] for i in range(texts.shape[1])})
    text_col = f'{FLD} {1} ' + df[0].astype(str) if mark_fields else df[0].astype(str)
    if sos_token is not None: text_col = f"{sos_token} " + text_col
    for i in range(1,len(df.columns)):
        #text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i]
        text_col += (f' {FLD} {i+1} ' if mark_fields else ' ') + df[i].astype(str)
    return text_col.values

In [0]:
path = Path(os.path.abspath(os.curdir))
train, val = [pd.read_csv(path / fname) for fname in ["train.csv", "valid.csv"]]

test = test_df

In [0]:
if config.testing:
    train = train.head(64)
    val = val.head(64)
    test = test.head(64)

In [0]:
fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))

In [0]:
fastai_tokenizer = Tokenizer(tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=config.max_seq_len), 
                             pre_rules=[], post_rules=[])

In [0]:
df.head()

,STORY,SECTION,Politics,Technology,Entertainment,Business
0,But the most painful was the huge reversal in ...,3,0.0,0.0,0.0,1.0
1,How formidable is the opposition alliance amon...,0,1.0,0.0,0.0,0.0
2,Most Asian currencies were trading lower today...,3,0.0,0.0,0.0,1.0
3,"If you want to answer any question, click on ‘...",1,0.0,1.0,0.0,0.0
4,"In global markets, gold prices edged up today ...",3,0.0,0.0,0.0,1.0


In [0]:
label_cols = ["Politics", "Technology", "Entertainment", 'Business']

In [0]:
databunch = TextDataBunch.from_df(".", train, val, test,
                   tokenizer=fastai_tokenizer,
                   vocab=fastai_bert_vocab,
                   include_bos=False,
                   include_eos=False,
                   text_cols="STORY",
                   label_cols=label_cols,
                   bs=config.bs,
                   collate_fn=partial(pad_collate, pad_first=False, pad_idx=0),
              )

In [0]:
class BertTokenizeProcessor(TokenizeProcessor):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer, include_bos=False, include_eos=False)

class BertNumericalizeProcessor(NumericalizeProcessor):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, vocab=Vocab(list(bert_tok.vocab.keys())), **kwargs)

def get_bert_processor(tokenizer:Tokenizer=None, vocab:Vocab=None):
    """
    Constructing preprocessors for BERT
    We remove sos/eos tokens since we add that ourselves in the tokenizer.
    We also use a custom vocabulary to match the numericalization with the original BERT model.
    """
    return [BertTokenizeProcessor(tokenizer=tokenizer),
            NumericalizeProcessor(vocab=vocab)]

class BertDataBunch(TextDataBunch):
    @classmethod
    def from_df(cls, path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:Optional[DataFrame]=None,
                tokenizer:Tokenizer=None, vocab:Vocab=None, classes:Collection[str]=None, text_cols:IntsOrStrs=1,
                label_cols:IntsOrStrs=0, label_delim:str=None, **kwargs) -> DataBunch:
        "Create a `TextDataBunch` from DataFrames."
        p_kwargs, kwargs = split_kwargs_by_func(kwargs, get_bert_processor)
        # use our custom processors while taking tokenizer and vocab as kwargs
        processor = get_bert_processor(tokenizer=tokenizer, vocab=vocab, **p_kwargs)
        if classes is None and is_listy(label_cols) and len(label_cols) > 1: classes = label_cols
        src = ItemLists(path, TextList.from_df(train_df, path, cols=text_cols, processor=processor),
                        TextList.from_df(valid_df, path, cols=text_cols, processor=processor))
        src = src.label_for_lm() if cls==TextLMDataBunch else src.label_from_df(cols=label_cols, classes=classes)
        if test_df is not None: src.add_test(TextList.from_df(test_df, path, cols=text_cols))
        return src.databunch(**kwargs)

In [0]:
bert_model = BertForSequenceClassification.from_pretrained(config.bert_model_name, num_labels=4)

100%|██████████| 1248501532/1248501532 [00:24<00:00, 51154187.81B/s]


In [0]:
loss_func = nn.BCEWithLogitsLoss()
from fastai.callbacks import CSVLogger

In [0]:
learner = Learner(databunch, bert_model,loss_func=loss_func, callback_fns=[partial(CSVLogger, append=True)])
if config.use_fp16: learner = learner.to_fp16()

In [0]:
learner.fit_one_cycle(4, 3e-5)

epoch,train_loss,valid_loss,time
0,0.071360,0.056850,06:15
1,0.054638,0.035068,06:20
2,0.012324,0.022944,06:21
3,0.010694,0.024125,06:18


In [0]:
def get_preds_as_nparray(ds_type) -> np.ndarray:
    """
    the get_preds method does not yield the elements in order by default
    we borrow the code from the RNNLearner to resort the elements into their correct order
    """
    preds = learner.get_preds(ds_type)[0].detach().cpu().numpy()
    #y = learner.get_preds(ds_type)[1].detach().cpu().numpy()
    
    sampler = [i for i in databunch.dl(ds_type).sampler]
    reverse_sampler = np.argsort(sampler)
    
    return preds[reverse_sampler, :]

In [0]:
preds = get_preds_as_nparray(DatasetType.Test)

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

idx = np.argmax(preds, axis=-1)
y_preds = np.zeros(preds.shape)
y_preds[np.arange(preds.shape[0]), idx] = 1

In [0]:
y_preds
preds= y_preds

In [0]:
dataset = pd.DataFrame({0:preds[:,0],1:preds[:,1],2:preds[:,2], 3:preds[:,3]})
s = pd.get_dummies(dataset).idxmax(1)
dataseti = pd.DataFrame({'SECTION':s})

In [0]:
dataseti.head()

,SECTION
0,1
1,2
2,1
3,1
4,1


In [0]:
dataseti.to_excel('submission.xlsx')

In [0]:
from google.colab import files
files.download('submission.xlsx')